In [1]:
from ftx_utilities import *
from ftx_snap_basis import *
import seaborn as sns
import plotly.express as px
import os
directory='C:/Users/david/Dropbox/mobilier/crypto/archived data/ftx/archived/'

In [2]:
futures_list=[from_parquet(directory+filename).sort_index(ascending=True) for filename in [filename for filename in os.listdir(directory) if "_futures" in filename]]

#### --------- show monthly max or premiums

In [16]:
def compute_premium(df: pd.DataFrame) -> pd.Series:
    return (df.filter(like='/mark/c').iloc[:,0]/df.filter(like='/indexes/close').iloc[:,0]-1)*10000
premium_list=[pd.DataFrame(data=compute_premium(df),columns=[str(df.columns[0]).split('/')[0]]) for df in futures_list]
symbol_list=[str(df.columns[0]).split('/')[0] for df in futures_list]

In [33]:
type(premium_list[2])

pandas.core.frame.DataFrame

In [ ]:
premium=premium_list[0]
for df in premium_list[1:]:
    premium=premium.join(df,how='outer')
premium=premium[~premium.index.duplicated()]

In [ ]:
ewM=premium.resample('1w').max()
ewM=ewM[~ewM.index.duplicated()]
fig = px.line(ewM)#, x='date', y="GOOG")
fig.show()

#### --------- premium hourly seasonality

In [ ]:
fundings=from_parquet(directory+'allfundings.parquet').filter(like=symbol).iloc[:,0].sort_index(ascending=True)

In [ ]:
twap= premium.rolling('1h').mean()/24
#fundings - twap[fundings.index]

In [ ]:
premium_vs_hour=premium-premium[premium.index.map(lambda t: t.replace(minute=0,second=0,microsecond=0))]
#sns.lineplot(x=premium_vs_hour.index.minute,y=premium_vs_hour.values)

In [ ]:
ewE=premium.ewm(times=premium.index,halflife=timedelta(days=2)).mean()
ewS=premium.ewm(times=premium.index,halflife=timedelta(days=2)).std()
ewZ=(premium-ewE)/ewS
#ewZ=ewZ.drop(index=ewZ.index[0])

In [ ]:
plotthat=premium
plotthat=plotthat[~plotthat.index.duplicated()]
last=plotthat[(plotthat.index>datetime.today()-timedelta(days=6))&(plotthat.index<datetime.today()-timedelta(days=5))]
#sns.lineplot(x=last.index,y=last)